# **RVC AI Cover Maker PT-BR**
Criado por [ShiromiyaG](https://github.com/ShiromiyaG)
- Colab inspirado no [AICoverGen](https://github.com/SociallyIneptWeeb/AICoverGen) por [SociallyIneptWeeb](https://github.com/SociallyIneptWeeb)
- Usa a versão do [blaise-tk](https://github.com/blaise-tk) do [RVC_CLI](https://github.com/blaise-tk/RVC_CLI)

In [ ]:
import tarfile, zipfile
import os
import sys
from google.colab import drive
from pathlib import Path
import contextlib

def extract_tar_file(tar_file_path, destination_path):
  if not Path(tar_file_path).exists():
      print(f"Tar file {tar_file_path} does not exist.")
      return

  extraction_failed = False

  with tarfile.open(tar_file_path, "r:gz") as tar:
      for member in tar.getmembers():
          try:
              tar.extract(member, destination_path)
          except Exception as e:
              print(f"Failed to extract file {member.name}: {e}")
              extraction_failed = True

  print(f"Extraction of {tar_file_path} to {destination_path} completed.")

  if os.path.exists(tar_file_path):
      try:
          os.remove(tar_file_path)
      except Exception as e:
          print(f"Failed to remove tar file {tar_file_path}: {e}")

  if extraction_failed:
      print("Extraction encountered errors.")

@contextlib.contextmanager
def supress_output(supress=True):
    if supress:
        with open(os.devnull, "w") as devnull:
            old_stdout = sys.stdout
            old_stderr = sys.stderr
            sys.stdout = devnull
            sys.stderr = devnull
            try:
                yield
            finally:
                sys.stdout = old_stdout
                sys.stderr = old_stderr
    else:
        yield

drive.mount('/content/drive')
Path('/content/dependencies').mkdir(parents=True, exist_ok=True)
os.chdir("/content")
# @title # **Instalar**
# @markdown ##### Sim, leva um tempo
# @markdown ##### Para achar bugs, tem a opção de não limpar o output
not_clean_outputs = False #@param {type:"boolean"}
# @markdown ##### Instalar dependencias pip precompiladas **(Pode estar datado)**
precompiled = True #@param {type:"boolean"}
with supress_output(not not_clean_outputs):
  !sudo apt-get install ffmpeg python3.10-venv aria2 libsamplerate0-dev -y
  !git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2.git
  if not precompiled:
    !pip install MVSEP-MDX23-Colab_v2/requirements.txt

def install_project(supress):
  with supress_output(supress):
    !wget -P /content "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/inference.py"
    !wget -P /content "https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/mdx23colab-0.1.0-py3-none-any.whl"
    !wget -P /content "https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/rvccli-0.1.0-py3-none-any.whl"
    !wget -P /content "https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/shiromiyautils-0.1.0-py3-none-any.whl"
    !pip install mdx23colab-0.1.0-py3-none-any.whl
    !pip install rvccli-0.1.0-py3-none-any.whl
    !pip install shiromiyautils-0.1.0-py3-none-any.whl

def install_precompiled_dependencies(supress=True):
  with supress_output(supress):
    arquivo_zip = '/content/RVCAICoverMakerDeps.tar.gz'
    pasta_destino = '/content/dependencies'
    Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
    !aria2c --console-log-level=error -x 16 -s 16 -k 1M https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/RVCAICoverMakerDeps.tar.gz -d /content -o RVCAICoverMakerDeps.tar.gz
    source_path = "/content/RVCAICoverMakerDeps.tar.gz"
    destination_path = "/"
    extract_tar_file(source_path, destination_path)

def install_dependencies(precompiled=True, supress=True):
  with supress_output(supress):
    if not precompiled:
      !pip install poetry pedalboard
      !poetry config virtualenvs.create false
      !pip install --no-deps pyrubberband dora-search retrying hydra-core>=1.1
      !pip install yt-dlp[default] wget git+https://github.com/IAHispano/gdown
      !pip install --upgrade --upgrade-strategy only-if-needed pydub soxr cmake audiofile samplerate==0.1.0
      !pip install python-dotenv samplerate==0.1.0

def setup_ultimatevocalremover_api(supress=True):
  with supress_output(supress):
    !git clone https://github.com/NextAudioGen/ultimatevocalremover_api.git
    !rm ultimatevocalremover_api/requirements.txt
    !rm ultimatevocalremover_api/src/models_dir/models.json
    !rm ultimatevocalremover_api/src/models_dir/mdx/modelparams/model_data.json
    !rm ultimatevocalremover_api/src/models_dir/vr_network/modelparams/model_data.json
    !wget -P ultimatevocalremover_api/src/models_dir/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/models.json"
    !wget -P ultimatevocalremover_api/src/models_dir/mdx/modelparams/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/mdx/model_data.json"
    !wget -P ultimatevocalremover_api/src/models_dir/vr_network/modelparams/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/vr/model_data.json"
    !wget -P ultimatevocalremover_api/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/requirements.txt"
    %cd ultimatevocalremover_api
    !pip install --upgrade --upgrade-strategy only-if-needed .
    %cd /content

def setup_kitware(supress=True):
  with supress_output(supress):
    !wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | gpg --dearmor - | sudo tee /etc/apt/trusted.gpg.d/kitware.gpg >/dev/null
    !sudo apt-add-repository "deb https://apt.kitware.com/ubuntu/ $(lsb_release -cs) main" -y
    !sudo apt update
    !sudo apt install kitware-archive-keyring
    !sudo rm /etc/apt/trusted.gpg.d/kitware.gpg
    !sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 6AF7F09730B3F0A4
    !sudo apt update
    !sudo apt-get install libsamplerate0-dev g++ make cmake

def setup_orpheusdl(precompiled=True, supress=True):
  with supress_output(supress):
    !git clone https://github.com/OrfiTeam/OrpheusDL.git
    if not precompiled:
      %cd OrpheusDL
      !poetry install --no-root -q > /dev/null 2>&1
      %cd ../
    !git clone https://git.ovosimpatico.com/ovosimpatico/orpheusdl-deezer.git ./OrpheusDL/modules/deezer
    !wget -P OrpheusDL/config "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/OrpheusDL/settings.json"

def setup_rvc_cli(precompiled=True, supress=True):
  with supress_output(supress):
    os.chdir("/content")
    !wget -P /content "https://github.com/blaise-tk/RVC_CLI/archive/refs/tags/1.1.1.zip"
    extraction_folder = os.path.join("/content")
    with zipfile.ZipFile("/content/1.1.1.zip", 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.rename("/content/RVC_CLI-1.1.1", "/content/RVC_CLI")
    %cd RVC_CLI
    if not precompiled:
      !poetry install --no-root > /dev/null 2>&1
    !python main.py prerequisites
    %cd /content

def main(precompiled=True,not_clean_outputs=False):
  if not not_clean_outputs:
    supress = True
  else:
    supress = False
  print("Installing dependencies...")
  with supress_output(supress):
    install_project(supress)
    install_dependencies(precompiled,supress)
    if precompiled:
      !pip uninstall numpy==1.26.4 -y
      !pip uninstall numpy==1.25.2 -y
      install_precompiled_dependencies(supress)
    else:
      setup_ultimatevocalremover_api(supress)
    setup_kitware(supress)
    setup_orpheusdl(precompiled, supress)
    setup_rvc_cli(precompiled, supress)
    !pip uninstall onnxruntime-gpu -y
    !python -m pip -q install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
    !pip install --upgrade --force-reinstall numpy==1.23.5
  print("Dependênicas instaladas")
  exit()

main(precompiled, not_clean_outputs)

# **Opcional** <font size="2">(Clique na seta oara mostrar)</font>

In [ ]:
#@title ### **Arquivo local**
from google.colab import files
import os
import fnmatch
from pathlib import Path
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
def search_files(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if file.endswith(".mp3") or file.endswith(".flac") or file.endswith(".wav"):
                caminho_arquivo = os.path.join(root, file)
                arquivos.append(caminho_arquivo)
    return arquivos
uploaded = files.upload()
uploaded = search_files("/content")
for arquivo in uploaded:
  !cp "{arquivo}" "/content/musicas/arquivos-originais"

In [ ]:
#@title ### **Arquivo do Google Drive**
drive_path = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}
import os
import fnmatch
from pathlib import Path
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)

def search_files(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if fnmatch.fnmatch(file, '*.mp3') or fnmatch.fnmatch(file, '*.flac') or fnmatch.fnmatch(file, '*.wav'):
                arquivos.append(os.path.join(root, file))
    return arquivos

arquivos_encontrados = search_files(drive_path)

for arquivo in arquivos_encontrados:
    !cp "{arquivo}" "/content/musicas/arquivos-originais"

In [ ]:
#@title ### **Faça download de uma música do YouTube ou  do Deezer**
# @markdown #### **Selecione uma das opções para fazer download**
link_of_yt = "" #@param {type:"string"}
link_of_deezer = "" #@param {type:"string"}
# @markdown ##### Se você vai usar o Deezer
bf_secret = "" #@param {type:"string"}
track_url_key = "" #@param {type:"string"}
arl = "" #@param {type:"string"}

if link_of_yt != "":
  !python inference.py download_yt --link "{link_of_yt}" --supress True
  clear_output()

if link_of_deezer != "":
  if bf_secret != "" and track_url_key != "" and arl != "":
    %cd OrpheusDL
    !python inference.py download_deezer --link "{link_of_deezer}" \
          --supress True \
          --bf_secret "{bf_secret}" \
          --track_url_key "{track_url_key}" \
          --arl "{arl}"
    if not_clean_outputs != True:
      clear_output()
    %cd ../
  else:
    raise ValueError("You need to provide the bf secret, track url key arl of your Deezer Premium account")

# **Programa**

In [ ]:
from pathlib import Path
import os
from glob import glob
from IPython.display import clear_output
from tqdm import tqdm
# @markdown ## **Selecione uma das opções para baixar uma música**
link_of_yt = "" #@param {type:"string"}
link_of_deezer = "" #@param {type:"string"}
# @markdown ##### *Se você for usar o Deezer para baixar*
bf_secret = "" #@param {type:"string"}
track_url_key = "" #@param {type:"string"}
arl = "" #@param {type:"string"}
# @markdown ---
# @markdown ## **Configurações do RVC**
rvc_model_link = ""  # @param {type:"string"}
rvc_model_name = os.path.splitext(os.path.basename(rvc_model_link))[0]
rvc_model_name_ext = os.path.splitext(os.path.basename(rvc_model_link))[1]
f0method = "rmvpe"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
pitch = 0  # @param {type:"slider", min:-24, max:24, step:0}
filter_radius = 3  # @param {type:"slider", min:0, max:10, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
autotune = False  # @param{type:"boolean"}
# @markdown ---
# @markdown ## **Opções da separação de vocais**
restore_gain_after_separation = False #@param {type:"boolean"}
filter_vocals_below_50hz = False #@param {type:"boolean"}
#@markdown  *Se você tem o Colab Pro e esta usando a v100 ou A100*
large_gpu = False # @param {type:"boolean"}
#@markdown  *Colocque BigShift=1 para  desativar*
BigShifts = 3 #@param {type:"slider", min:1, max:41, step:1}
BSRoformer_model = 'ep_317_1297' #@param ["ep_317_1297", "ep_368_1296"]
weight_BSRoformer = 10 #@param {type:"slider", min:0, max:10, step:1}
overlap_BSRoformer = 0.25 #@param {type:"slider", min:0, max:0.95, step:0.05}
use_InstVoc = True #@param {type:"boolean"}
weight_InstVoc = 4 #@param {type:"slider", min:0, max:10, step:1}
overlap_InstVoc = 0.25 #@param {type:"slider", min:0, max:0.95, step:0.05}
use_VitLarge = True #@param {type:"boolean"}
weight_VitLarge = 1 #@param {type:"slider", min:0, max:10, step:1}
overlap_VitLarge = 0.25 #@param {type:"slider", min:0, max:0.95, step:0.05}
use_VOCFT = False #@param {type:"boolean"}
weight_VOCFT = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_VOCFT = 0.25 #@param {type:"slider", min:0, max:0.95, step:0.05}
# @markdown ---
# @markdown ## **Escolha algoritmos de Ensemble** (junção de espectograma)
# @markdown ##### *O Ensemble leva mais tempo, mas a qualidade é maior*
Instrumental_Ensemble = True #@param {type:"boolean"}
algorithm_ensemble_inst =  "Max Spec" # @param ['averege', 'Max Spec', 'Min Spec'] {allow-input: false}
# @markdown ---
# @markdown ## **Efeitos dos vocais**
# @markdown ### Reverb
reverb = True # @param{type:"boolean"}
reverb_size = 0.15  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_wetness = 0.2 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_dryness = 0.8 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_damping = 0.7 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_width = 1.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
# @markdown ### Limiter
limiter = False # @param{type:"boolean"}
limiter_threshold_db = -6.0 # @param {type:"slider", min:-60.0, max:0.0, step:0.1}
limiter_release_ms = 60.0 # @param {type:"slider", min:1.0, max:5000.0, step:0.1}
limiter_ceiling_db = -1.0 # @param {type:"slider", min:-60.0, max:0.0, step:0.1}
# @markdown ### Compressor
compressor = False # @param{type:"boolean"}
compressor_ratio = 1.0 # @param {type:"slider", min:1.0, max:20.0, step:0.0}
compressor_threshold_db = -45.0 # @param {type:"slider", min:-60.0, max:0.0, step:0.1}
compressor_attack_ms = 2.0 # @param {type:"slider", min:0.1, max:200.0, step:0.1}
compressor_release_ms = 50.0 # @param {type:"slider", min:1.0, max:5000.0, step:0.1}
# @markdown ---
# @markdown ## **Opções de mixagem do áudio**
# @markdown ### Ganho
Voc_Vol = -2 # @param {type:"slider", min:-10, max:10, step:0}
Inst_Vol = 0 # @param {type:"slider", min:-10, max:10, step:0}
# @markdown ### Limiter
limiter_master = False # @param{type:"boolean"}
limiter_threshold_db_master = -6.0 # @param {type:"slider", min:-60.0, max:0.0, step:0.1}
limiter_release_ms_master = 60.0 # @param {type:"slider", min:1.0, max:5000.0, step:0.1}
# @markdown ### Ruído do RVC
remove_noise_from_RVC = False  # @param{type:"boolean"}
noise_db_limit = -35 # @param {type:"slider", min:-50, max:0, step:0.1}
# @markdown ---
# @markdown ## **Opções do Colab**
# @markdown ##### *Suprimir Outputs*
supress = True # @param{type:"boolean"}
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

input_folder = "/content/musicas/arquivos-originais"
no_back_folder = "/content/musicas/sem-back"
no_inst_folder = "/content/musicas/sem-intrumental"
output_vocals_folder = "/content/musicas/output-vocals-folder"
model_dir = "/content/arquivos"
stage1_dir = "/content/musicas/inst-etapa1"
stage2_dir = "/content/musicas/inst-etapa2"
final_output_dir = "/content/musicas/output_inst"
model_destination_folder = "/content/RVC_CLI/logs"
output_rvc = "/content/output_rvc"
output_reverb_folder = "/content/output_reverb"
output_mix_folder = "/content/output_mix"
output_drive = "/content/drive/MyDrive/RVCAICoverMakerOutput"
logs = "/content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs"
folders = [input_folder, no_back_folder, no_inst_folder, output_vocals_folder, model_dir, stage1_dir, stage2_dir, final_output_dir, model_destination_folder, output_rvc, output_reverb_folder, output_mix_folder, output_drive, logs]

for folder in folders:
    path = Path(folder)
    if not path.exists():
        path.mkdir(parents=True, exist_ok=True)
    elif path.is_file():
        print(f"{folder} is a file, not a directory.")

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

def find_files(directory, extensions):
  files = glob(f'{directory}/**/*{extensions}', recursive=True)
  return files[0]

if link_of_yt != "":
  !python inference.py download_yt --link "{link_of_yt}" --supress True
  clear_output()

if link_of_deezer != "":
  if bf_secret != "" and track_url_key != "" and arl != "":
    %cd OrpheusDL
    !python inference.py download_deezer --link "{link_of_deezer}" \
          --supress True \
          --bf_secret "{bf_secret}" \
          --track_url_key "{track_url_key}" \
          --arl "{arl}"
    if not_clean_outputs != True:
      clear_output()
    %cd ../
  else:
    raise ValueError("You need to provide the bf secret, track url key arl of your Deezer Premium account")

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Procesando", len(input_files), "arquivo" if len(input_files) == 1 else "arquivos")
  basename = os.path.basename(input_file).split(".")[0]
  !python inference.py separate_vocals \
          --input_file "{input_file}" \
          --output_final_folder {output_vocals_folder}\
          --no_inst_folder "{no_inst_folder}" \
          --no_back_folder "{no_back_folder}" \
          --output_folder "{output_intermediary_folder}" \
          --device "{device}" \
          --supress {supress} \
          --large_gpu {large_gpu} \
          --single_onnx False \
          --cpu False \
          --overlap_VOCFT {overlap_VOCFT} \
          --overlap_VitLarge {overlap_VitLarge} \
          --overlap_InstVoc {overlap_InstVoc} \
          --overlap_BSRoformer {overlap_BSRoformer} \
          --weight_InstVoc {weight_InstVoc} \
          --weight_VOCFT {weight_VOCFT} \
          --weight_VitLarge {weight_VitLarge} \
          --weight_BSRoformer {weight_BSRoformer} \
          --BigShifts {BigShifts} \
          --vocals_only True \
          --use_BSRoformer True \
          --BSRoformer_model {BSRoformer_model} \
          --use_InstVoc {use_InstVoc} \
          --use_VitLarge {use_VitLarge} \
          --use_VOCFT {use_VOCFT} \
          --restore_gain {restore_gain} \
          --filter_vocals {filter_vocals} 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_voc.txt
  output_vocals = get_last_modified_file("/content/musicas/output-vocals-folder")
  !cp "{output_vocals}" "{output_drive}"

  !python inference.py separate_instrumentals \
          --input_file "{input_file}" \
          --instrumental_ensemble {Instrumental_Ensemble} \
          --algorithm_ensemble_inst "{algorithm_ensemble_inst}" \
          --stage1_dir "{stage1_dir}" \
          --stage2_dir "{stage2_dir}" \
          --final_output_dir "{final_output_dir}" \
          --device "{device}" \
          --supress {supress} \
          --language "BR" 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_inst.txt
  output_instrumentals = get_last_modified_file("/content/musicas/output_inst")
  !cp "{output_instrumentals}" "{output_drive}"

  %cd RVC_CLI
  !python main.py download \
          --model_link "{rvc_model_link}" 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_rvc_download.txt

  pth_file = find_files("/content/RVC_CLI/logs", ".pth")
  index_file = find_files("/content/RVC_CLI/logs", ".index")
  output_rvc = "/content/output_rvc/output_rvc.wav"

  !python main.py infer \
          --input_path "{output_vocals}" \
          --output_path "{output_rvc}" \
          --f0up_key {pitch} \
          --filter_radius {filter_radius} \
          --index_rate {index_rate} \
          --hop_length {hop_length} \
          --rms_mix_rate {rms_mix_rate} \
          --protect {protect} \
          --pth_path {pth_file} \
          --index_path {index_file} \
          --f0autotune {autotune} \
          --f0method "{f0method}" \
          --split_audio {split_audio} \
          --clean_audio {clean_audio} \
          --clean_strength {clean_strength} \
          --export_format "WAV" 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_rvc.txt
  %cd ..
  output_rvc = get_last_modified_file("/content/output_rvc")
  !cp "{output_rvc}" "{output_drive}"

  !python inference.py reverb \
          --audio_path "{output_rvc}" \
          --reverb {reverb} \
          --reverb_size {reverb_size} \
          --reverb_wetness {reverb_wetness} \
          --reverb_dryness {reverb_dryness} \
          --reverb_damping {reverb_damping} \
          --reverb_width {reverb_width} \
          --limiter {limiter} \
          --limiter_threshold_db {limiter_threshold_db} \
          --limiter_release_ms {limiter_release_ms} \
          --compressor {compressor} \
          --compressor_ratio {compressor_ratio} \
          --compressor_threshold_db {compressor_threshold_db} \
          --compressor_attack_ms {compressor_attack_ms} \
          --compressor_release_ms {compressor_release_ms} \
          --output_path "{output_reverb_folder}/{basename}_output_reverb.wav" \
          --supress {supress} \
          --language "BR" 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_reverb.txt
  output_reverb = get_last_modified_file("/content/output_reverb")
  !cp "{output_reverb}" "{output_drive}"

  if remove_noise_from_RVC:
    !python inference.py remove_noise \
          --audio_path "{output_reverb}" \
          --noise_db_limit {noise_db_limit} \
          --output_path "{output_reverb_folder}/{basename}_output_reverb.wav" \
          --supress {supress} \
          --language "BR" 2>> log_noise.txt
    output_reverb = get_last_modified_file("/content/output_reverb")
    !cp "{output_reverb}" "{output_drive}"

  !python inference.py mix_audio \
          --input_file "{basename}" \
          --vocals_path "{output_reverb}" \
          --inst_path "{output_instrumentals}" \
          --output_path "{output_mix_folder}" \
          --rvc_model_name "{rvc_model_name}" \
          --main_gain {Voc_Vol} \
          --inst_gain {Inst_Vol} \
          --output_format "flac" \
          --supress {supress} \
          --language "BR" 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_mix.txt

  if limiter_master:
    !python inference.py reverb \
            --audio_path "{output_rvc}" \
            --reverb False \
            --reverb_size {reverb_size} \
            --reverb_wetness {reverb_wetness} \
            --reverb_dryness {reverb_dryness} \
            --reverb_damping {reverb_damping} \
            --reverb_width {reverb_width} \
            --limiter {limiter_master} \
            --limiter_threshold_db {limiter_threshold_db_master} \
            --limiter_release_ms {limiter_release_ms_master} \
            --compressor False \
            --compressor_ratio {compressor_ratio} \
            --compressor_threshold_db {compressor_threshold_db} \
            --compressor_attack_ms {compressor_attack_ms} \
            --compressor_release_ms {compressor_release_ms} \
            --output_path "{output_mix_folder}/{basename}_output_mix_limiter.wav" \
            --supress {supress} \
            --language "BR" 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_reverb_master.txt
  output_mix = get_last_modified_file("/content/output_mix")
  !cp "{output_mix}" "{output_drive}"
  print(f"O arquivo FLAC esta na pasta \"RVCAICoverMakerOutput\" no seu Drive!")